In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from datetime import datetime

import html
import re
import nltk
from unicodedata import normalize
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [3]:
review = pd.read_json('/content/drive/MyDrive/reviews_Digital_Music_5.json', lines = True)


In [4]:
review.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,1200528000,"01 17, 2008"


**EDA**

In [5]:
print(review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64706 entries, 0 to 64705
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      64706 non-null  object
 1   asin            64706 non-null  object
 2   reviewerName    64529 non-null  object
 3   helpful         64706 non-null  object
 4   reviewText      64706 non-null  object
 5   overall         64706 non-null  int64 
 6   summary         64706 non-null  object
 7   unixReviewTime  64706 non-null  int64 
 8   reviewTime      64706 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.4+ MB
None


In [6]:
# We want to switch the unixReviewTime to the accurate time when each review was posted.
switch_time = lambda row: datetime.fromtimestamp(row).strftime("%m-%d-%Y")
review["unixReviewTime"] = review["unixReviewTime"].apply(switch_time)



In [7]:
# Hence that, we don't need the column reviewTime anymore and we would use drop function. 

review.drop(labels="reviewTime", axis=1, inplace=True)
display(review.head())

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,09-12-2006
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,06-03-2001
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,07-14-2003
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,05-03-2000
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,01-17-2008


In [8]:
# Each review has to store as string in the reviewText series. We choose the course number as a sample product review.

print(review["reviewText"].iloc[5430])

Truth be told? This album probably represented the very first funk by a contemporary artist I ever heard. Keep in mind it was when it came out. And at the time I had no idea what a musical genre (let alone funk) even was. The music of Cameo has always had a strong attraction to me ever since-likely due to that core musical memory. Historically for Cameo,this was an interesting time. Starting with 1984'sShe's Strange,Cameo pared down to a trio of three members in bandleader/founder Larry Blackson on lead vocals and bass with Nathan Leftenant and Tomi Jenkins as vocalists. Charlie Singleton left the band functionally to start a solo career. Yet the deepest thing about that was that Charlie,along with other members of the band,didn't leave completely. He,along with session musicians such as the Brecker brothers remained behind on this album which,as it were wound up being their iconic breakthrough album commercially-at least as far as pop char success was concerned.The title song and "Can

In [9]:
# We also want to make sure there is no negative rating in the review and all of the points are from [5-1]

print(review.overall.unique())

[5 4 3 1 2]


In [10]:
# Extract the root word by using WordNetLemmatizer from the Natural Language Toolkit
# We first lower all of the reviews, punctuations and stopwords. 
 
pattern = r"\&\#[0-9]+\;"

review["preprocessed"] = review["reviewText"].str.replace(pat=pattern, repl="", regex=True)

In [11]:
%%time
import re
import nltk

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet

#import nltk resources
resources = ["wordnet", "stopwords", "punkt", \
             "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"]

for resource in resources:
    try:
        nltk.data.find("tokenizers/" + resource)
    except LookupError:
        nltk.download(resource)

#create Lemmatizer object
lemma = WordNetLemmatizer()

def lemmatize_word(tagged_token):
    """ Returns lemmatized word given its tag"""
    root = []
    for token in tagged_token:
        tag = token[1][0]
        word = token[0]
        if tag.startswith('J'):
            root.append(lemma.lemmatize(word, wordnet.ADJ))
        elif tag.startswith('V'):
            root.append(lemma.lemmatize(word, wordnet.VERB))
        elif tag.startswith('N'):
            root.append(lemma.lemmatize(word, wordnet.NOUN))
        elif tag.startswith('R'):
            root.append(lemma.lemmatize(word, wordnet.ADV))
        else:          
            root.append(word)
    return root

def lemmatize_doc(document):
    """ Tags words then returns sentence with lemmatized words"""
    lemmatized_list = []
    tokenized_sent = sent_tokenize(document)
    for sentence in tokenized_sent:
        no_punctuation = re.sub(r"[`'\",.!?()]", " ", sentence)
        tokenized_word = word_tokenize(no_punctuation)
        tagged_token = pos_tag(tokenized_word)
        lemmatized = lemmatize_word(tagged_token)
        lemmatized_list.extend(lemmatized)
    return " ".join(lemmatized_list)

#apply our functions
review["preprocessed"] = review["preprocessed"].apply(lambda row: lemmatize_doc(row))

print(review["preprocessed"].iloc[5430])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/maxent_treebank_pos_tagger.zip.
Truth be tell This album probably represent the very first funk by a contemporary artist I ever hear Keep in mind it be when it come out And at the time I have no idea what a musical genre let alone funk even be The music of Cameo have always have a strong attraction to me ever since-likely due to that core musical memory Historically for Cameo this be

In [12]:
# Removing the Accents (e.g. from naïve to naive)


from unicodedata import normalize

remove_accent = lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")

review["preprocessed"] = review["preprocessed"].apply(remove_accent)

print(review["preprocessed"].iloc[5430])

Truth be tell This album probably represent the very first funk by a contemporary artist I ever hear Keep in mind it be when it come out And at the time I have no idea what a musical genre let alone funk even be The music of Cameo have always have a strong attraction to me ever since-likely due to that core musical memory Historically for Cameo this be an interesting time Starting with 1984 sShe s Strange Cameo par down to a trio of three member in bandleader/founder Larry Blackson on lead vocal and bass with Nathan Leftenant and Tomi Jenkins as vocalist Charlie Singleton leave the band functionally to start a solo career Yet the deep thing about that be that Charlie along with other member of the band didn t leave completely He along with session musician such as the Brecker brother remain behind on this album which as it be wind up be their iconic breakthrough album commercially-at least as far as pop char success be concern The title song and Candy be of course the signature mid 80 

In [13]:
# Removing punctuations 


pattern = r"[^\w\s]"

review["preprocessed"] = review["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

print(review["preprocessed"].iloc[5430])

Truth be tell This album probably represent the very first funk by a contemporary artist I ever hear Keep in mind it be when it come out And at the time I have no idea what a musical genre let alone funk even be The music of Cameo have always have a strong attraction to me ever since likely due to that core musical memory Historically for Cameo this be an interesting time Starting with 1984 sShe s Strange Cameo par down to a trio of three member in bandleader founder Larry Blackson on lead vocal and bass with Nathan Leftenant and Tomi Jenkins as vocalist Charlie Singleton leave the band functionally to start a solo career Yet the deep thing about that be that Charlie along with other member of the band didn t leave completely He along with session musician such as the Brecker brother remain behind on this album which as it be wind up be their iconic breakthrough album commercially at least as far as pop char success be concern The title song and Candy be of course the signature mid 80 

In [14]:
# lower all of the characters
review["preprocessed"] = review["preprocessed"].str.lower()

print(review["preprocessed"].iloc[5430])

truth be tell this album probably represent the very first funk by a contemporary artist i ever hear keep in mind it be when it come out and at the time i have no idea what a musical genre let alone funk even be the music of cameo have always have a strong attraction to me ever since likely due to that core musical memory historically for cameo this be an interesting time starting with 1984 sshe s strange cameo par down to a trio of three member in bandleader founder larry blackson on lead vocal and bass with nathan leftenant and tomi jenkins as vocalist charlie singleton leave the band functionally to start a solo career yet the deep thing about that be that charlie along with other member of the band didn t leave completely he along with session musician such as the brecker brother remain behind on this album which as it be wind up be their iconic breakthrough album commercially at least as far as pop char success be concern the title song and candy be of course the signature mid 80 

In [15]:
# Dropping the extra space between words and make sure, there is only a single whitespace between words.

pattern = r"[\s]+"

review["preprocessed"] = review["preprocessed"].str.replace(pat=pattern, repl=" ", regex=True)

print(review["preprocessed"].iloc[5430])

truth be tell this album probably represent the very first funk by a contemporary artist i ever hear keep in mind it be when it come out and at the time i have no idea what a musical genre let alone funk even be the music of cameo have always have a strong attraction to me ever since likely due to that core musical memory historically for cameo this be an interesting time starting with 1984 sshe s strange cameo par down to a trio of three member in bandleader founder larry blackson on lead vocal and bass with nathan leftenant and tomi jenkins as vocalist charlie singleton leave the band functionally to start a solo career yet the deep thing about that be that charlie along with other member of the band didn t leave completely he along with session musician such as the brecker brother remain behind on this album which as it be wind up be their iconic breakthrough album commercially at least as far as pop char success be concern the title song and candy be of course the signature mid 80 

In [16]:
# In order to build up the corpora and we would break down the document words into individual words and tokens.

corpora = review["preprocessed"].values
tokenized = [corpus.split(" ") for corpus in corpora]

print(tokenized[5430])

['truth', 'be', 'tell', 'this', 'album', 'probably', 'represent', 'the', 'very', 'first', 'funk', 'by', 'a', 'contemporary', 'artist', 'i', 'ever', 'hear', 'keep', 'in', 'mind', 'it', 'be', 'when', 'it', 'come', 'out', 'and', 'at', 'the', 'time', 'i', 'have', 'no', 'idea', 'what', 'a', 'musical', 'genre', 'let', 'alone', 'funk', 'even', 'be', 'the', 'music', 'of', 'cameo', 'have', 'always', 'have', 'a', 'strong', 'attraction', 'to', 'me', 'ever', 'since', 'likely', 'due', 'to', 'that', 'core', 'musical', 'memory', 'historically', 'for', 'cameo', 'this', 'be', 'an', 'interesting', 'time', 'starting', 'with', '1984', 'sshe', 's', 'strange', 'cameo', 'par', 'down', 'to', 'a', 'trio', 'of', 'three', 'member', 'in', 'bandleader', 'founder', 'larry', 'blackson', 'on', 'lead', 'vocal', 'and', 'bass', 'with', 'nathan', 'leftenant', 'and', 'tomi', 'jenkins', 'as', 'vocalist', 'charlie', 'singleton', 'leave', 'the', 'band', 'functionally', 'to', 'start', 'a', 'solo', 'career', 'yet', 'the', 'dee

In [17]:
# To be considered a phrase, the number of times that two words should appear next to each other is set to at least 500.
# For the threshold I would choose 50,because the higher threshold means the more often two words must appera adjacent to be grouped.
from gensim.models import Phrases
from gensim.models.phrases import Phraser

bi_gram = Phrases(tokenized, min_count=300, threshold=50)

tri_gram = Phrases(bi_gram[tokenized], min_count=300, threshold=50)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [18]:
# Unigrams are single piece of tokens and I print first 100 as a sample.
uni_gram_tokens = set([token for text in tokenized for token in text])
uni_gram_tokens = set(filter(lambda x: x != "", uni_gram_tokens))

print(list(uni_gram_tokens)[:100])

['opry', 'prlly', 'shack', 'freestyled', 'thewheels', 'conditional', 'empty3', 'apology', 'acception', 'relatively', 'hormonal', 'company', '2013s', 'lor', 'unawareendless', 'panzzy', 'erazed', 'sunlight', 'confirmen', 'mostconsistant', 'onsound', 'fipped', 'voacally', 'haunteda', 'foray', 'frombegin', 'heaven4', 'freud', 'problemz', 'bongo', 'revised', 'inanity', 'vthe', 'sypda', 'tillermanandteaser', 'gg', 'mendler', 'pensive', 'slickly', 'ballpoint', 'everyoneis', 'god7', 'recallother', 'songsmithing', '4988005518354shm', 'intimidate', 'anathallo', 'earthiest', 'consequence', 'mash', 'oustanding', 'frusciante', 'soleil', 'jodys', 'chatartic', 'sylviaaaa', 'abetted', 'auch', 'thatweren', 'stoned', '6x9', 'unparellel', 'sensilbilities', 'fritgerald', 'antagonist', 'shabba', 'telly', 'diestel', 'kalage', 'coup', 'cinderlla', 'hitthe', 'coquettish', 'russo', 'productive', 'lumiere', 'addittional', 'difnitly', 'futures', 'earchecks', 'generica', 'brotherly', 'haymaker', 'psychosexual', '

In [19]:
# Bi_gram  are consists of two pieces of tokens and I print first 100 as a sample.
bigram_min = bi_gram.min_count

bi_condition = lambda x: x[1] >= bigram_min

bi_gram_tokens = dict(filter(bi_condition, bi_gram.vocab.items()))
bi_gram_tokens = set([token.decode("utf-8") \
                      for token in bi_gram_tokens])

bi_grams_only = bi_gram_tokens.difference(uni_gram_tokens)
print(list(bi_grams_only)[:100])

['', 'tire_of', 'but_for', 'all_three', 'fan_be', 'slow_and', 'his_fan', 'album_feature', 'be_amazing', 'it_when', '1980_s', 'stand_the', 'not_get', 'make_his', 'know_what', 'someone_who', 'song_ever', 'the_back', 'song_as', 'and_an', 'you_should', 'that_do', 'no_doubt', 'part_of', 't_do', 'at_its', 'the_thing', 'on_some', 'that_we', 'a_young', 'you_that', 'and_let', 'need_to', 'really_like', 'all_i', 'full_length', 'at_it', 'want_to', 'sound_very', 'why_be', 'talk_about', 'me_of', 'song_it', 'the_instrument', 'record_the', 'most_of', 'majority_of', 'ice_cube', 'really_a', 'that_he', 'fan_i', 'after_listen', 'not_too', 'the_line', 'which_make', 'first_i', 'paul_mccartney', 'or_be', 'because_he', 'but_what', 'when_i', 'video_be', 'her_best', 'it_also', 'have_for', 'on_me', 'way_that', 'my_eye', 'the_test', 'especially_the', 'this_have', 'it_come', 'music_that', 'probably_my', 'album_at', 'greatest_hits', 'track_it', 'be_play', 'weren_t', 'nate_dogg', 'do_so', 'favorite_album', 'do_a', '

In [20]:
# Trigram  are consists of three pieces of tokens and I print first 100 as a sample.
trigram_min = tri_gram.min_count

tri_condition = lambda x: x[1] >= trigram_min

tri_gram_tokens = dict(filter(tri_condition, tri_gram.vocab.items()))
tri_gram_tokens = set([token.decode("utf-8") \
                       for token in tri_gram_tokens])

tri_grams_only = tri_gram_tokens.difference(bi_gram_tokens)
print(list(tri_grams_only)[:50])


['hip_hop_album', 'r_amp_b', 'mary_j_blige', 'rock_n_roll', 'in_hip_hop', 'gon_na_be', 'the_hip_hop', 'be_gon_na', 'highly_recommend_this', 'the_r_b', 'the_beach_boys', 'the_white_stripes', 'i_highly_recommend', 'you_wan_na', 'the_moody_blues', 'the_liner_note', 'wan_na_be', 'i_wan_na', 'a_hip_hop', 'the_west_coast', 'of_hip_hop', 'hip_hop_fan', 'jay_z_s', 'hip_hop_and']


In [21]:
# We applied tri_gram and bi_gram phrasers to our tokenized corpora.
# tokenized = [Phraser(tri_gram)[Phraser(bi_gram)[i]] for i in tokenized]

In [22]:
# Single-character tokens are removed from every tokenized document we don't
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized]

print(tokenized[5430])

['truth', 'be', 'tell', 'this', 'album', 'probably', 'represent', 'the', 'very', 'first', 'funk', 'by', 'contemporary', 'artist', 'ever', 'hear', 'keep', 'in', 'mind', 'it', 'be', 'when', 'it', 'come', 'out', 'and', 'at', 'the', 'time', 'have', 'no', 'idea', 'what', 'musical', 'genre', 'let', 'alone', 'funk', 'even', 'be', 'the', 'music', 'of', 'cameo', 'have', 'always', 'have', 'strong', 'attraction', 'to', 'me', 'ever', 'since', 'likely', 'due', 'to', 'that', 'core', 'musical', 'memory', 'historically', 'for', 'cameo', 'this', 'be', 'an', 'interesting', 'time', 'starting', 'with', '1984', 'sshe', 'strange', 'cameo', 'par', 'down', 'to', 'trio', 'of', 'three', 'member', 'in', 'bandleader', 'founder', 'larry', 'blackson', 'on', 'lead', 'vocal', 'and', 'bass', 'with', 'nathan', 'leftenant', 'and', 'tomi', 'jenkins', 'as', 'vocalist', 'charlie', 'singleton', 'leave', 'the', 'band', 'functionally', 'to', 'start', 'solo', 'career', 'yet', 'the', 'deep', 'thing', 'about', 'that', 'be', 'tha

In [23]:
# We created a vocabulary for key-value pairs from every product review. The first 5 words are belowed.
from gensim.corpora.dictionary import Dictionary

vocabulary = Dictionary(tokenized)

vocabulary_keys = list(vocabulary.token2id)[20:50]

for key in vocabulary_keys:
    print(f"ID: {vocabulary.token2id[key]}, Token: {key}")

ID: 20, Token: but
ID: 21, Token: come
ID: 22, Token: creative
ID: 23, Token: dark
ID: 24, Token: day
ID: 25, Token: decide
ID: 26, Token: deorum
ID: 27, Token: disconcert
ID: 28, Token: don
ID: 29, Token: elegaic
ID: 30, Token: end
ID: 31, Token: enya
ID: 32, Token: etc
ID: 33, Token: fan
ID: 34, Token: find
ID: 35, Token: for
ID: 36, Token: from
ID: 37, Token: great
ID: 38, Token: hard
ID: 39, Token: have
ID: 40, Token: her
ID: 41, Token: hold
ID: 42, Token: home
ID: 43, Token: hope
ID: 44, Token: hopeful
ID: 45, Token: if
ID: 46, Token: in
ID: 47, Token: it
ID: 48, Token: last
ID: 49, Token: like


In [24]:
# For the classic method, we just find the frequency of how many times that word appear most. 
bow = [vocabulary.doc2bow(doc) for doc in tokenized]

for idx, freq in bow[0]:
    print(f"Word: {vocabulary.get(idx)}, Frequency: {freq}")

Word: 11, Frequency: 1
Word: 1995, Frequency: 1
Word: about, Frequency: 2
Word: age, Frequency: 2
Word: ago, Frequency: 1
Word: agree, Frequency: 1
Word: album, Frequency: 2
Word: all, Frequency: 1
Word: always, Frequency: 1
Word: amarantine, Frequency: 1
Word: and, Frequency: 1
Word: any, Frequency: 1
Word: anywhere, Frequency: 1
Word: at, Frequency: 1
Word: back, Frequency: 1
Word: be, Frequency: 12
Word: before, Frequency: 1
Word: believe, Frequency: 1
Word: bittersweet, Frequency: 1
Word: blend, Frequency: 1
Word: but, Frequency: 1
Word: come, Frequency: 1
Word: creative, Frequency: 1
Word: dark, Frequency: 1
Word: day, Frequency: 1
Word: decide, Frequency: 1
Word: deorum, Frequency: 1
Word: disconcert, Frequency: 2
Word: don, Frequency: 1
Word: elegaic, Frequency: 1
Word: end, Frequency: 1
Word: enya, Frequency: 3
Word: etc, Frequency: 1
Word: fan, Frequency: 1
Word: find, Frequency: 1
Word: for, Frequency: 2
Word: from, Frequency: 2
Word: great, Frequency: 1
Word: hard, Frequency

In [25]:
#Term Frequency-Inverse Document Frequency approach is a method that assigns continuous values instead of simple integers for the token frequency.
from gensim.models.tfidfmodel import TfidfModel

tfidf = TfidfModel(bow)

for idx, weight in tfidf[bow[5430]]:
    print(f"Word: {vocabulary.get(idx)}, Weight: {weight:.3f}")

Word: about, Weight: 0.024
Word: album, Weight: 0.021
Word: all, Weight: 0.021
Word: always, Weight: 0.021
Word: and, Weight: 0.017
Word: any, Weight: 0.017
Word: at, Weight: 0.020
Word: back, Weight: 0.030
Word: be, Weight: 0.015
Word: but, Weight: 0.004
Word: come, Weight: 0.039
Word: fan, Weight: 0.015
Word: find, Weight: 0.018
Word: for, Weight: 0.020
Word: from, Weight: 0.015
Word: hard, Weight: 0.020
Word: have, Weight: 0.023
Word: in, Weight: 0.021
Word: it, Weight: 0.025
Word: like, Weight: 0.006
Word: look, Weight: 0.022
Word: memory, Weight: 0.038
Word: music, Weight: 0.038
Word: of, Weight: 0.035
Word: on, Weight: 0.019
Word: out, Weight: 0.036
Word: pop, Weight: 0.036
Word: song, Weight: 0.009
Word: sound, Weight: 0.032
Word: that, Weight: 0.053
Word: the, Weight: 0.022
Word: this, Weight: 0.011
Word: time, Weight: 0.021
Word: title, Weight: 0.020
Word: to, Weight: 0.014
Word: up, Weight: 0.021
Word: well, Weight: 0.046
Word: who, Weight: 0.043
Word: with, Weight: 0.037
Wor

In [26]:
# We set the minimum word to 1 and embed every token in the corpora. 
%%time
import numpy as np
from gensim.test.utils import common_texts
from gensim.models import word2vec

np.set_printoptions(suppress=True)

feature_size = 20
context_size = 5
min_word = 1

word_vec= word2vec.Word2Vec(tokenized, size=feature_size, \
                            window=context_size, min_count=min_word, \
                            seed=5430)

CPU times: user 2min 53s, sys: 882 ms, total: 2min 54s
Wall time: 1min 41s


In [27]:
# We take five common words in our corpora and see the similarity word comes from their word_bank counterpart.
word_bank = ["music", "hiphop", "excitability", "hits", "lyricl"]

for word in word_bank[:]:
    related_vec = word_vec.wv.most_similar(word, topn=5)
    related_words = np.array(related_vec)[:,0]
    word_bank.extend(related_words)
    print(f"{word}: {related_words}")

music: ['genre' 'formatting' 'element' 'listener' 'muse']
hiphop: ['hardcore' 'westcoast' 'rap' 'theory' 'southern']
excitability: ['themost' 'pharrels' 'tempering' 'tricc' 'scruggscleveland']
hits: ['compilation' 'albums' 'anthology' 'blueprint' 'cds']
lyricl: ['tuapc' 'antarctic' 'zawadzki' 'brutus' 'oklahoman']


In [28]:
# We also want to see some examples of this vector on Word2Vec based on the similarities to figure out top one closest word relationship.
word_vec.wv.most_similar(positive=["peace", "love"], \
                      negative=[], topn=2)

[('tried', 0.7892420291900635), ('is', 0.7598125338554382)]

In [29]:
word_vec.wv.most_similar(positive=["love"], \
                      negative=["hate"], topn=2)

[('thetechnology', 0.7378014326095581), ('10diamonds', 0.7335427403450012)]

In [30]:
word_vec.wv.most_similar(positive=["quality"], \
                      negative=["cheap"], topn=2)

[('musicianship', 0.8440196514129639), ('piece', 0.8145103454589844)]

**Predict Score**

**Sentiment Flask implement**

In [31]:
from flask import Flask, request, render_template
import json
import pandas as pd
import csv

app = Flask(__name__)

@app.route('/')
def my_form():
    return render_template("my-form.html")

@app.route('/', methods=['POST'])
def my_form_post():
    val = request.form['userinput']
    result=[]
    #read sentiment analysis result from csv
    with open('/content/drive/MyDrive/5430/5430_project/Digital_sentiment.csv', newline='') as csvfile:
        spamreader = csv.DictReader(csvfile)
        for row in spamreader:
            if val in row['preprocessed']:
                result.append(row['sentiment'])
    #return the summary of sentiment analysis given keywords  
    return "Sentiment Analysis---Positive: "+str(result.count('positive'))+", Neutral: "+str(result.count('neutral'))+", Negative: "+str(result.count('negative'))+", Positive Rate: "+str(result.count('positive')/(result.count('positive')+result.count('neutral')+result.count('negative')))



In [32]:
app.run(host='localhost', port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001/ (Press CTRL+C to quit)


**LDA**

In [32]:
 text = review[['reviewText','summary']]
 #Create a dataframe that contains only the 'preprocessed' column

In [33]:
text

,reviewText,summary
0,"It's hard to believe ""Memory of Trees"" came ou...",Enya's last great album
1,"A clasically-styled and introverted album, Mem...",Enya at her most elegant
2,I never thought Enya would reach the sublime h...,The best so far
3,This is the third review of an irish album I w...,Ireland produces good music.
4,"Enya, despite being a successful recording art...",4.5; music to dream to
...,...,...
64701,I like the reggae sound a lot in this song. I ...,Cool song
64702,I first heard this on Sirius and had to have i...,Great Song
64703,"I absolutely love this song, it downloaded fin...",Five Stars
64704,"Reggae, island beats aren't really my cup of t...",Well-crafted song


In [34]:
key_word = 'rock music'
#Define the search key word

In [35]:
#Filter the review that contains the key word
key_word_text = text.loc[text['reviewText'].str.contains(key_word), "summary"]

In [36]:
key_word_text.to_csv('text_1')
#save the key_word_text to a csv file 

In [37]:
df = pd.read_csv('text_1')
# read the csv file of key word to a dataframe

In [38]:
df

,Unnamed: 0,summary
0,85,Holy jeez this is good!
1,98,My All-Time Favorite Album Period!
2,101,Going Through The Emotions
3,156,Wilson's great album
4,169,One of the Greatest Beach Boys Albums
...,...,...
649,64016,"Boring. Don't get me wrong, I was a HUGE fan."
650,64035,Okkervil River - A full workout
651,64112,Brothers of the head
652,64311,Killers Alternative/Indie Rock Singles


#### Library imports and utility text cleanup function

In [39]:
import requests, json, re
import pandas as pd
import matplotlib
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')
# import string library function 
import string 

stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation) 

def text_cleanup(input_text):
    one = " ".join([i for i in input_text.lower().split() if i not in stop_words])
    two = "".join(re.sub(r'[^a-zA-Z ]', '', i) for i in one if i not in punctuation)
    three = [WordNetLemmatizer().lemmatize(i) for i in two.split()]
    return three
    #define a text clean up function to clean the text of of input text with stop word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Train LDA topic model and display topic distribution

In [1]:
#import the necessary function to train the lda model
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 34.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 56 kB/s 
     |████████████████████████████████| 11.3 MB 31.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=124872bd4d31005d316e44a43c0fe446a195d3558164b8841f6aafbcc998a452
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [40]:
nltk.download('wordnet')
#download 'wordnet' package

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
#import pyLDAvis.gensim
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
#enable the pyldavis 


titles = df[['summary']].applymap(text_cleanup)['summary']
#apply the text_cleanuo to the summary column of the df
dictionary = Dictionary(titles)
#create a dict to contain thetitles
dictionary.filter_extremes(no_below=10, no_above=0.75)
#set min and max limits for the document frequency achieve the best result
corpora = [dictionary.doc2bow(doc) for doc in titles]

# Running and Training LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=10, id2word = dictionary, passes=50)
#the number of topics is set to 10, which means 10 topic clusters are displayed

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [42]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpora, dictionary, R =10)
#R is set to 10 so that 10 topic clusters are displayed
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.094184  0.251434       1        1  17.206696
7     -0.193260 -0.044751       2        1  14.395674
8     -0.296565  0.109627       3        1  13.223984
5     -0.218998  0.094090       4        1  10.226438
4      0.088784 -0.203343       5        1   9.143873
1      0.271919  0.162237       6        1   9.121332
3     -0.126442 -0.038388       7        1   8.420525
0      0.073250 -0.268759       8        1   7.936328
2      0.221380  0.109762       9        1   5.451924
6      0.085747 -0.171908      10        1   4.873226, topic_info=           Term       Freq      Total Category  logprob  loglift
8          rock  67.000000  67.000000  Default  10.0000  10.0000
7       classic  31.000000  31.000000  Default   9.0000   9.0000
5   masterpiece  23.000000  23.000000  Default   8.0000   8.0000
2         great  32.000000  32.000000  Default   7.0000   7.0000
17        music  30.000000  30.000000  Default   6.0000   6.0000
..          ...        ...        ...      ...      ...      ...
7       classic   0.188508  31.616148  Topic10  -4.9626  -2.1009
13        debut   0.188483  16.559858  Topic10  -4.9627  -1.4543
8          rock   0.188476  67.748845  Topic10  -4.9627  -2.8632
10         best   0.188470  35.003316  Topic10  -4.9628  -2.2028
17        music   0.188470  30.360930  Topic10  -4.9628  -2.0606

[157 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1         1  0.147009        album
1         2  0.120280        album
1         3  0.481120        album
1         4  0.173738        album
1         7  0.066822        album
22        5  0.864574         back
12        8  0.952454         band
10        2  0.371393         best
10        3  0.599943         best
16        8  0.956246       better
15        1  0.676039           cd
15        2  0.061458           cd
15        5  0.184374           cd
7         7  0.980512      classic
13        1  0.966192        debut
11        2  0.957723         ever
0         5  0.948266         good
2         4  0.974452        great
3         2  0.337952     greatest
3         8  0.608313     greatest
5        10  0.969983  masterpiece
19        4  0.933171        moody
17        6  0.955175        music
4         2  0.972178          one
18        2  0.162402          pop
18        5  0.730807          pop
9         5  0.235268       record
9         7  0.705803       record
8         1  0.811822         rock
8         6  0.103323         rock
8         9  0.073802         rock
14        3  0.942020            s
6         6  0.897581        sound
21        9  0.957906         star
20        2  0.941230         time, R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 8, 9, 6, 5, 2, 4, 1, 3, 7])

**10 Topic taxonomy**

### Taxonomy-Guided Semantic Similarity Classifier

In [45]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

model_path = '/content/drive/MyDrive/'
#set model path for the wordnet for the future import

#### Load the pre-trained Word2Vec model

In [46]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model
    #define a function for load the wordvec model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec model...


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The b

Finished loading Word2Vec model...


/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  weights = fromstring(fin.read(binary_len), dtype=REAL).astype(datatype)
/usr/local/lib/python3.7/dist-packages/gensim/models/utils_any2vec.py:212: DeprecationWarning: The binary mode of fromstring is deprecated, as it behav

### Define taxonomy of topics (classes) and keywords

In [47]:
topic_taxonomy = {
    "Song":
    #define the topic taxonomy based on the results from the clustered 10 most salient terms
    {
        "Hopeful": "is a hopeful song,looking to possibilities",
        "Burst Song": "a couple of songs that absolutely burst with an expected raw power",
        "Great Song": "Brilliant early album full of great songs",
        "Song Writer":"Status as one of the great songwriters"
    },
    "Band":
    {
        "Support": "A brilliant, original voice with stellar band support",
        "Band members": "its band member's progressive roots"
    },
    "Album":
    {
        "80s": "The Best album of the 80s, hands down",
        "Favourite Ambum": "Quite possibly my favorite rock album ever recorded."
    },
    "Good":
    {
        "Good stuff": "more good stuff besides the hits",
        "Remix good": "One of the few remix albums that are any good"
    },
    "Rock":
    {
        "Rock Album": "Excellent modern heavy art rock/fusion album",
        "Righteous Rocker": "The Best Album From A Righteous Rocker",
        "Rock Fan":"Belongs on Every Rock Fan's Shelf"
    },
    "Track":
    {
        "Sond Track": "The soundtrack of the mid 70's to late 80's",
        "strong bonus tracks": "A good album enhanced by strong bonus tracks"
    },
    "Love":
    {
        "Lover's Voice": "A Lover's Voice Fires Mountain Sides",
        "Love of Music": "Love of Music Showed in Everything We Heard"
    },
    "Cd":
    {
        "Reissue": "Great Moody Blues Reissue CD",
        "beautiful CD": "A really beautiful CD"
    },
    "Guitar":
    {
        "acoustic guitar": "Haunting pieces played on acoustic guitar",
        "Guitar Review": "Grammy Award Winning All-Star Blues Guitar Review"
    },
    "Beat":
    {
        "basic beats": "basic beats, less fury",
        "Hard Beats": "Return Of The Real Hard Beats And Real Rap  {5 Stars}"
    }                
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Check if the input term exists in the model vocabulary

In [48]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

#### Calculate semantic similarity of a pair of strings

In [49]:
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    #define function for calculating the similarities between 2 set of vectors
    output = vectors.n_similarity(s1words, s2words)
    return output

#### Classify topics (run similarity of input string against taxonomy, sort and return top 3 results)

In [50]:
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            #define a function to classify topics
            #input is the input text that is being classified
            #vectors are pre-defined
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [66]:
#ignore deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#classify the input text to a specific topic
output1 = classify_topics('First and still best album from this prolific but controversial musician', model_word2vec)
print(output1)




[('Favourite Ambum', 0.7224968075752258), ('Remix good', 0.680170476436615), ('Great Song', 0.6601774096488953)]


In [67]:
#classify the input text to a specific topic
output2 = classify_topics('Nick Drake sings songs and plays guitar on his final album', model_word2vec)
print(output2)

[('Great Song', 0.7173271775245667), ('acoustic guitar', 0.6982833743095398), ('Favourite Ambum', 0.6521052718162537)]
